In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from layers.QConv2D import QConv2D
from circuits.random import ry_random
from utils.plotter import *
from data.datahandler import datahandler
from data.datareader import datareader

import matplotlib.pyplot as plt
import pennylane as qml
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os

## Load dataset

In [ ]:
dataset_name = 'EuroSAT'
root = os.path.join('datasets', dataset_name)
dhandler = datahandler(root)
dhandler.print_report(name=dataset_name)

In [ ]:
labels_mapper, x, y = dhandler.unpack(dhandler.paths)

print('Dataset Size')
print('Images   --- {}'.format(len(x)))

print('\nTraining Dataset samples')
print('X Train --- {} '.format(x[0]))
print('Y Train --- {} '.format(y[0]))

In [ ]:
data, metadata = datareader.load(x[0])

In [ ]:
(xi, yi) = next(iter(datareader.generator((x, y), 16, (64,64,3))))

In [ ]:
count = np.zeros(len(y[0]))
for i, (xi,yi,pi) in tqdm(enumerate(datareader.generatorv2((x, y), (64,64,3)))):
    if i >= len(x):
        break
    
    count[np.argmax(yi)] += 1

yy = len(y[0])
xx = np.arange(yy)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(yy, 3))
ax.bar(xx, count)
for i in range(len(count)):
    ax.text(x=i, y = count[i], s= count[i], c='black')
ax.set_xticks(xx)
ax.set_xticklabels(labels = dhandler.paths.keys(), rotation=90)
plt.show()
plt.close()

## Define Quantum Circuit 

In [ ]:
QUBITS      = 16
KERNEL_SIZE = 3
FILTERS     = 16
N_LAYERS    = 1
STRIDE      = 2
NUM_JOBS    = 8

In [ ]:
circuit = ry_random(QUBITS, KERNEL_SIZE, FILTERS, N_LAYERS)

In [ ]:
drawer = qml.draw(circuit)
print(drawer(np.random.rand(QUBITS)))

## Quantum Convolutional 2D layer

In [ ]:
conv1 = QConv2D(
    ry_random(QUBITS, KERNEL_SIZE, FILTERS, N_LAYERS),
    FILTERS, 
    KERNEL_SIZE, 
    STRIDE, 
    NUM_JOBS
)

In [ ]:
(xi, yi) = next(iter(datareader.generator((x, y), 1, (64,64,3))))
img = xi[0]
out1 = conv1.apply(img, verbose = True)

In [ ]:
print('Image shape       ', img.shape)
print('QuaConv2D L1 shape', out1.shape)

In [ ]:
plot_result(img, out1)

## Quantum Preprocessing of the dataset

In [ ]:
# Create folder structures
root2 = root.replace(dataset_name, dataset_name+'_processed')
for path in dhandler.paths.keys():
    os.makedirs(os.path.join(root2, path), exist_ok=True)

In [ ]:
gen = iter(datareader.generatorv2((x, y), (64,64,3)))

for i in tqdm(range(len(x))):
    (xi, yi, pi) = next(gen)
    out1 = conv1.apply(xi, verbose=True)
    pi = pi.replace(dataset_name, dataset_name+'_processed')
    pi = pi.replace('.jpg', '.npy')
    
    with open(pi, 'wb') as f:
        np.save(f, out1)